## Webscraping Committee of adjustments website
This file imports the webscraped data from the coa portal so appropriate missing middle buildings can be picked out and their drawings downloaded for takeoff.<br>

#### IMPORTS: 


In [3]:
# Data
import numpy as np
import scipy as sp
import pandas as pd

#Viz
import matplotlib.pyplot as plt
import seaborn as sns

#os
import os
import glob

plt.style.use('ggplot')
sns.set_context('notebook')
sns.set(font='Arial')

#### Import the data from .csv

In [93]:
# Create a function for concatenating all files, cleaning repeated values.
def import_webscraped_data(path):
    """
    Imports the webscraped data from .csv files in a folder, removes duplicates.
    """

    files = glob.glob(os.path.join(path, "*.csv"))
    
    # concat, rmv duplicates
    df = pd.concat((pd.read_csv(f, index_col=0) for f in files))
    df = df.drop_duplicates().reset_index(drop=True)
    
    print('[INFO] complete.')
    
    return df      

In [94]:
mypath = r'C:/Users/Keagan Rankin/OneDrive - University of Toronto/Saxe - Rankin/Missing Middle/Missing Middle Code & Results/mm webscraping/scraped data/'

s_data = import_webscraped_data(mypath)

[INFO] complete.


In [95]:
s_data = s_data.drop_duplicates(subset='link').drop_duplicates(subset='description').dropna(subset='description').reset_index(drop=True)

In [96]:
s_data

,address,ward,application number,application type,date submitted,status,description,link
0,35 admiral rd,ward 11: university-rosedale,21 157248 ste 11 mv,minor variance,19/05/2021,tlab appeal,to alter the existing three-storey dwelling by...,http://app.toronto.ca/AIC/index.do?folderRsn=d...
1,6 admiral rd,ward 11: university-rosedale,21 159268 ste 11 mv,minor variance,22/05/2021,postponed,to alter the existing two-storey semi-detached...,http://app.toronto.ca/AIC/index.do?folderRsn=H...
2,1109 bathurst st,ward 11: university-rosedale,20 159922 ste 11 oz,rezoning,24/06/2020,omb appeal,zoning by-law amendment application for a prop...,http://app.toronto.ca/AIC/index.do?folderRsn=8...
3,26 albany ave,ward 11: university-rosedale,22 127776 ste 11 mv,minor variance,28/03/2022,hearing scheduled,to construct a new ancillary structure (two-ca...,http://app.toronto.ca/AIC/index.do?folderRsn=4...
4,275 albany ave,ward 11: university-rosedale,15 256733 ste 20 oz,opa & rezoning,25/11/2015,closed,application for official plan amendment and re...,http://app.toronto.ca/AIC/index.do?folderRsn=t...
...,...,...,...,...,...,...,...,...
243,804 shaw st,ward 11: university-rosedale,22 109048 ste 11 mv,minor variance,31/01/2022,accepted,alter the existing semi-detached dwelling by c...,http://app.toronto.ca/AIC/index.do?folderRsn=p...
244,998 shaw st,ward 11: university-rosedale,22 129407 ste 11 mv,minor variance,1/4/2022,tentatively scheduled,to alter the existing two-storey detached dwel...,http://app.toronto.ca/AIC/index.do?folderRsn=b...
245,57 sylvan ave,ward 9: davenport,21 249724 ste 09 mv,minor variance,17/12/2021,closed,to convert the existing two-storey building fr...,http://app.toronto.ca/AIC/index.do?folderRsn=T...
246,19 wallace ave,ward 9: davenport,22 108537 ste 09 mv,minor variance,28/01/2022,hearing scheduled,to alter the existing two-storey detached dwel...,http://app.toronto.ca/AIC/index.do?folderRsn=e...


#### Promising low rise/missing middle buildings using a corpus search.

In [101]:
# Look for these kinds of buildings.
mm_corpus = ['semi', 'laneway', 'row house', 'three storey', 
             '2-story', 'third storey', 'two-storey', 'addition', 'mixed-use', 
             'duplex', 'triplex', 'townhouse', 'low-rise', 'low rise']

# Ignore buildings that are taller than 2 storeys
num1 = ["{}-storey".format(i) for i in range(5,100)]
num2 = ["{} storey".format(i) for i in range(5,100)]
mm_negate_corpus = num1 + num2

# This could be improved: we don't just want or, we also want some ands in there as well.
corp_data= s_data[s_data['description'].str.contains('|'.join(mm_corpus))]
corp_data= corp_data[~corp_data['description'].str.contains('|'.join(mm_negate_corpus))].reset_index(drop=True)

for i in range(corp_data.shape[0]):
    print(corp_data.index[i])
    print(corp_data['description'].iloc[i])
    print('')


0
to alter the existing two-storey semi-detached dwelling by constructing a third storey addition, two-storey rear addition, front porch and rear deck.

1
to construct a new ancillary structure (two-car detached garage) in the rear yard of the existing four-unit, semi-detached dwelling to replace two existing rear parking spaces.

2
to alter the existing two-storey detached dwelling by constructing a rear two-storey addition and rear second storey additions as well as two-storey side additions.

3
to sever existing lot with semi detached building into two lots.

4
to construct a new three-storey detached dwelling with a ground floor deck, a second floor balcony and a third floor terrace. also, to permit a front yard parking pad. the existing two-storey dwelling will be demolished.

5
to alter the existing three-storey semi-detached mixed-use building by constructing a partial third storey addition and a partially covered rooftop patio.

6
to obtain consent to convey for the purpose of 

In [100]:
o_path = "C:/Users/Keagan Rankin/OneDrive - University of Toronto/Saxe - Rankin/Missing Middle/Missing Middle Code & Results/mm webscraping/annex_mm_buildings.csv"
corp_data.to_csv(o_path, mode='a', header=not os.path.exists(path))